In [3]:
from Utilities import *
from PuzzlePiece import *

In [4]:
# gather all the pieces
pieces = {}
for p in ['C','N','NE','E','SE','S','SW','W','NW']:
    pieces[p] = Piece('representations/' + p + '_border_ordered.csv')

In [5]:
# get the first piece
query = pieces.pop('C')

In [8]:
# find the closest match in the other pieces
mx_score = 0
mx_piece = ''

for p in pieces:
    # need an alignment object to store the results from these 
    SuffTable, mx, mx_Q, mx_T, length = \
        AlignPieces_Euclid(query.sampled_pts, pieces[p].sampled_pts, window = 10)
    
    sims = []
    for i in range(1,len(SuffTable)):
        for j in range(1,len(SuffTable[i])):
            sims.append(SuffTable[i][j])

    sims.sort()
    cutoff = sims[int(len(sims)*.1)]
    
    print(p, min(sims))
    
    # could build the dynamic programming table here
    SuffTable, mx, mx_Q, mx_T, length = \
        AlignPieces_Euclid(query.sampled_pts, pieces[p].sampled_pts,
                           mode = 'SW align', window = 10, cutoff = cutoff)
    
    print(p, mx)
    if mx > mx_score:
        mx_score = mx
        mx_piece = p
        
print('Best: ', mx_piece, mx_score)

N 4.69041575982343
N 88
NE 7.542245505072464
NE 26
E 6.24964657564981
E 77
SE 4.385894205005382
SE 90
S 6.082762530298219
S 39
SW 5.825711306448937
SW 61
W 7.123429878844059
W 42
NW 4.898979485566356
NW 55
Best:  SE 90


In [ ]:
# access the alignment object instead of reacalculating
SuffTable, mx, mx_q, mx_t, length = \
        AlignPieces_Euclid(query.sampled_pts, pieces[mx_piece].sampled_pts, window = 10)

sims = []
for i in range(1,len(SuffTable)):
    for j in range(1,len(SuffTable[i])):
        sims.append(SuffTable[i][j])

sims.sort()
cutoff = sims[int(len(sims)*.1)]
    
SuffTable, mx, mx_q, mx_t, length = \
        AlignPieces_Euclid(query.sampled_pts, pieces[mx_piece].sampled_pts,
                           mode = 'SW align', window = 10, cutoff = cutoff)

print(mx_piece, mx)
target = pieces.pop(mx_piece)

In [ ]:
#find the best matched point - this is the end of the best match window
Q_pt = query.sampled_pts[list(query.sampled_pts.keys())[mx_q-1]]
T_pt = target.sampled_pts[list(target.sampled_pts.keys())[::-1][mx_t-1]] 

# find x,y shift to align pieces
Q_xshift = query.sampled_pts[Q_pt]['x'] - target.sampled_pts[T_pt]['x']
Q_yshift = query.sampled_pts[Q_pt]['y'] - target.sampled_pts[T_pt]['y'] 

plt.figure(figsize = [10,10])
plt.scatter( [query.sampled_pts.sampled_pts[k]['x'] - Q_xshift for k,v in query.sampled_pts.items() ],
             [Q_pts[k]['y'] - Q_yshift for k,v in query.sampled_pts.items() ])
plt.scatter( [T_pts[k]['x'] for k,v in target.sampled_pts.items() ],
             [T_pts[k]['y'] for k,v in target.sampled_pts.items() ])

plt.plot( [query.sampled_pts[Q_pt]['x'] - Q_xshift, target.sampled_pts[T_pt]['x']],
          [query.sampled_pts[Q_pt]['y'] - Q_yshift, target.sampled_pts[T_pt]['y']],
        linewidth = 3)

# show best matched points - second will overplot first due to alignment
plt.scatter( query.sampled_pts[Q_pt]['x'] - Q_xshift, query.sampled_pts[Q_pt]['y'] - Q_yshift, s=225)
plt.scatter( target.sampled_pts[T_pt]['x'], target.sampled_pts[T_pt]['y'], s=196 ) 

# show similarity window
# the black and white points repsent the points in the positive scoring diagonal
# of the suffix table starting at the maximum scoring point
# these points are the best locally aligned points
# Q_window = [Q_order_pts[p] for p in list(Q_order_pts.keys())[ max((mx_q-length),0):(mx_q)] ]
# plt.scatter( [Q_pts[p]['x'] - Q_xshift for p in Q_window],
#              [Q_pts[p]['y'] - Q_yshift for p in Q_window], c = 'white' )
# T_window = [T_order_pts[p] for p in list(T_order_pts.keys())[::-1][(mx_t-length):(mx_t)] ]
# plt.scatter( [T_pts[p]['x'] for p in T_window],
#              [T_pts[p]['y'] for p in T_window], c = 'black' )

In [26]:
print(Q_pt)

{'orig_idx': 1045, 'x': 7, 'y': 144}
